In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os, sys
import pandas as pd
import PIL.Image
from datetime import datetime

root = '/usr/local/serenceslab/maggie/shapeDim/'

sys.path.append(os.path.join(root, 'Analysis'))
from code_utils import file_utils, data_utils, plot_utils

#### Count all runs in scanner sessions

In [2]:
df = pd.DataFrame(columns=['subject','session','num_main_runs','num_main_trials',\
                           'num_rep_runs','num_rep_trials',\
                           'num_silh_runs'])
                           
df_allsess = pd.DataFrame(columns=['subject','num_main_runs','num_main_trials',\
                           'num_rep_runs','num_rep_trials',\
                           'num_silh_runs'])
                           
                           

subjects = np.arange(1,8)

xx = -1
for si, ss in enumerate(subjects):
    
    bdat = pd.read_csv(os.path.join(root, 'DataBehavior','S%02d'%ss, 'S%02d_maintask_preproc_all.csv'%ss), \
                       index_col = 0)
    bdat_rep = pd.read_csv(os.path.join(root, 'DataBehavior','S%02d'%ss, 'S%02d_reptask_preproc_all.csv'%ss), \
                       index_col = 0)
    
    n_silh_count = 0
    n_silh_trial_count = 0
    for ses in [1,2,3]:
        
        bdir = os.path.join(root, 'DataBehavior','S%02d'%ss, 'Session%d'%ses)
        silh_files = os.listdir(bdir)
        silh_files = [ff for ff in silh_files if 'Silh' in ff]
        if len(silh_files)>0:
            TheData = file_utils.load_mat_behav_data(os.path.join(bdir, silh_files[0]))
            n_silh_runs = len(TheData)
            n_silh_trials = n_silh_runs*TheData[0]['p']['nTrials']
            # da = TheData[0]['t']['TheDate']
            # da = datetime.strptime(da, '%y%m%d')
            # da = datetime.strftime(da, '%D')
        else:
            n_silh_runs = int(0)
            n_silh_trials = int(0)
            # da = ''
            
        n_silh_count+=n_silh_runs
        n_silh_trial_count+=n_silh_trials
        
        n_main_runs = len(np.unique(bdat['run_overall'][bdat['sess']==ses]))
        n_main_trials = np.shape(bdat['run_overall'][bdat['sess']==ses])[0]
        
        n_rep_runs = len(np.unique(bdat_rep['run_overall'][bdat_rep['sess']==ses]))
        n_rep_trials = np.shape(bdat_rep['run_overall'][bdat_rep['sess']==ses])[0]
        
        xx+=1
        df = df.append(pd.DataFrame({'subject': ss, \
                                     'session': ses, \
                                     # 'date': da, \
                                     'num_main_runs': n_main_runs,\
                                     'num_main_trials': n_main_trials, \
                                     'num_rep_runs': n_rep_runs,\
                                     'num_rep_trials': n_rep_trials, \
                                     'num_silh_runs': n_silh_count,\
                                     'num_silh_trials': n_silh_trials, \
                                     }, index=[xx]))
      
    n_main_runs = len(np.unique(bdat['run_overall']))
    n_main_trials = np.shape(bdat)[0]

    n_rep_runs = len(np.unique(bdat_rep['run_overall']))
    n_rep_trials = np.shape(bdat_rep)[0]

    df_allsess = df_allsess.append(pd.DataFrame({'subject': ss, \
                                     'num_main_runs': n_main_runs,\
                                     'num_main_trials': n_main_trials, \
                                     'num_rep_runs': n_rep_runs,\
                                     'num_rep_trials': n_rep_trials, \
                                     'num_silh_runs': n_silh_count,\
                                     'num_silh_trials': n_silh_trial_count, \
                                     }, index=[xx]))
        

In [3]:
df_allsess

,subject,num_main_runs,num_main_trials,num_rep_runs,num_rep_trials,num_silh_runs,num_silh_trials
2,1,36,1728,12,576,5,100.0
5,2,36,1728,12,576,6,120.0
8,3,36,1728,12,576,6,120.0
11,4,36,1728,12,576,5,100.0
14,5,36,1728,12,576,7,140.0
17,6,33,1584,11,528,6,120.0
20,7,36,1728,7,336,4,80.0


In [4]:
df

,subject,session,num_main_runs,num_main_trials,num_rep_runs,num_rep_trials,num_silh_runs,num_silh_trials
0,1,1,12,576,4,192,0,0.0
1,1,2,12,576,4,192,2,40.0
2,1,3,12,576,4,192,5,60.0
3,2,1,12,576,4,192,2,40.0
4,2,2,12,576,4,192,4,40.0
5,2,3,12,576,4,192,6,40.0
6,3,1,12,576,4,192,2,40.0
7,3,2,12,576,4,192,4,40.0
8,3,3,12,576,4,192,6,40.0
9,4,1,12,576,3,144,1,20.0


#### Count runs in training sessions 

In [12]:
df_training = pd.DataFrame(columns=['subject',
                                    'num_main_runs','num_main_trials',\
                                    'num_maintrain_runs','num_maintrain_trials',\
                           'num_rep_runs','num_rep_trials',\
                           'num_silh_runs', 'num_silh_trials',\
                           'num_pretask_runs', 'num_pretask_trials'])
                                                  
subjects = np.arange(1,8)

xx = -1
for si, ss in enumerate(subjects):
    
    bdir = os.path.join(root, 'DataBehavior','S%02d'%ss,'Training')
    bfiles = os.listdir(bdir)
    
    # print(bfiles)
    
    
    silh_files = [ff for ff in bfiles if 'Silh' in ff]
    if len(silh_files)>0:
        TheData = file_utils.load_mat_behav_data(os.path.join(bdir, silh_files[0]))
        n_silh_runs = len(TheData)
        n_silh_trials = n_silh_runs*TheData[0]['p']['nTrials']
    else:
        n_silh_runs = int(0)
        n_silh_trials = int(0)
        
    rep_files = [ff for ff in bfiles if 'OneBack' in ff]
    if len(rep_files)>0:
        TheData = file_utils.load_mat_behav_data(os.path.join(bdir, rep_files[0]))
        n_rep_runs = len(TheData)
        n_rep_trials = n_rep_runs*TheData[0]['p']['nTrials']
    else:
        n_rep_runs = int(0)
        n_rep_trials = int(0)
        
    pretask_files = [ff for ff in bfiles if 'PreTask' in ff]
    if len(pretask_files)>0:
        TheData = file_utils.load_mat_behav_data(os.path.join(bdir, pretask_files[0]))
        n_pretask_runs = len(TheData)
        n_pretask_trials = n_pretask_runs*TheData[0]['p']['nTrials']
    else:
        n_pretask_runs = int(0)
        n_pretask_trials = int(0)
        
    main_files = [ff for ff in bfiles if 'Main' in ff and 'TRAINING' not in ff]
    main_run_count = 0; main_trial_count = 0;
    if len(main_files)>0:
        for mf in main_files:
            TheData = file_utils.load_mat_behav_data(os.path.join(bdir, mf))
            n_main_runs = len(TheData)
            n_main_trials = n_main_runs*TheData[0]['p']['nTrials']
            main_run_count+=n_main_runs
            main_trial_count+=n_main_trials
    else:
        main_run_count=0
        main_trial_count=0
        
    main_files = [ff for ff in bfiles if 'Main' in ff and 'TRAINING' in ff]
    maintrain_run_count = 0; maintrain_trial_count = 0;
    if len(main_files)>0:
        for mf in main_files:
            TheData = file_utils.load_mat_behav_data(os.path.join(bdir, mf))
            n_main_runs = len(TheData)
            n_main_trials = n_main_runs*TheData[0]['p']['nTrials']
            maintrain_run_count+=n_main_runs
            maintrain_trial_count+=n_main_trials
    else:
        maintrain_run_count=0
        maintrain_trial_count=0
        
    xx+=1
    df_training = df_training.append(pd.DataFrame({'subject': ss, \
                                     'num_main_runs': main_run_count, \
                                     'num_main_trials': main_trial_count, \
                                     'num_maintrain_runs': maintrain_run_count, \
                                     'num_maintrain_trials': maintrain_trial_count, \
                                     'num_rep_runs': n_rep_runs,\
                                     'num_rep_trials': n_rep_trials, \
                                     'num_silh_runs': n_silh_runs,\
                                     'num_silh_trials': n_silh_trials, \
                                     'num_pretask_runs': n_pretask_runs,\
                                     'num_pretask_trials': n_pretask_trials, \
                                     }, index=[xx]))
        

In [13]:
df_training

,subject,num_main_runs,num_main_trials,num_maintrain_runs,num_maintrain_trials,num_rep_runs,num_rep_trials,num_silh_runs,num_silh_trials,num_pretask_runs,num_pretask_trials
0,1,9,432,5,120,1,48,1,20,1,48
1,2,12,576,6,30,0,0,0,0,0,0
2,3,12,576,9,45,0,0,0,0,0,0
3,4,13,624,7,35,0,0,0,0,0,0
4,5,12,576,6,30,0,0,0,0,0,0
5,6,12,576,8,40,0,0,0,0,0,0
6,7,12,576,6,30,0,0,0,0,0,0


#### Find dates/times of each session

In [15]:
from datetime import datetime

df = pd.DataFrame(columns=['subject','session','date', 'time', 'init'])
                           
subjects = np.arange(1,8)

xx = -1
for si, ss in enumerate(subjects):
    
    for ses in ['Training', 'Session1','Session2','Session3']:
        
        bdir = os.path.join(root, 'DataBehavior','S%02d'%ss, ses)
        main_files = os.listdir(bdir)
        main_files = [ff for ff in main_files if ('Main' in ff)]
        dates = []; inits = []; times = []
        for file in main_files:
            
            TheData = file_utils.load_mat_behav_data(os.path.join(bdir, main_files[0]))
            
            da = [TheData[ii]['t']['TheDate'] for ii in range(len(TheData))]
            da = [datetime.strftime(datetime.strptime(d, '%y%m%d'), '%D') for d in da]
            
            ini = [TheData[ii]['p']['SubInit'] for ii in range(len(TheData))]
            
            tim = [TheData[ii]['t']['TimeStamp'] for ii in range(len(TheData))]
            tim = [datetime.strftime(datetime.strptime(t, '%H%M'), '%I:%M %p') for t in tim]
            
            dates += da
            inits += ini
            times += tim
            
        # print(np.unique(dates), np.unique(inits))
        # print(np.unique(times))
       
        xx+=1
        df = df.append(pd.DataFrame({'subject': ss, \
                                     'session': ses, \
                                     'date': da[0], \
                                     'init': inits[0], \
                                     'time': times[0], \
                                     }, index=[xx]))
      

In [17]:
df

,subject,session,date,time,init
0,1,Training,04/06/21,04:20 PM,JD
1,1,Session1,04/09/21,05:19 PM,CP
2,1,Session2,04/14/21,05:11 PM,CP
3,1,Session3,04/15/21,05:26 PM,CP
4,2,Training,06/25/21,03:32 PM,AC
5,2,Session1,07/02/21,04:08 PM,BX
6,2,Session2,07/03/21,12:01 PM,BX
7,2,Session3,07/10/21,03:53 PM,BX
8,3,Training,07/03/21,10:47 AM,BR
9,3,Session1,07/03/21,04:26 PM,BR


In [18]:
# How many days apart are training/scanning?

days = [3, 7, 0, 3, 3, 1, 16]
np.mean(days), np.min(days), np.max(days)

(4.714285714285714, 0, 16)